In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from os import path
from fastai.vision.all import *
from fastai.callback.tracker import SaveModelCallback

In [ ]:
set_seed(999)

In [ ]:
dataset_path = Path('../input/cassava-leaf-disease-classification')
os.listdir(dataset_path)

In [ ]:
train_df = pd.read_csv(dataset_path/'train.csv')

In [ ]:
train_df['path'] = train_df['image_id'].map(lambda x:dataset_path/'train_images'/x)
train_df = train_df.drop(columns=['image_id'])
train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
train_df = train_df[['path','label']]
train_df.head(10)

In [ ]:
item_tfms = RandomResizedCrop(460, min_scale=0.75, ratio=(1.,1.))
batch_tfms = [*aug_transforms(size=448, max_warp=0), Normalize.from_stats(*imagenet_stats)]
bs=24

In [ ]:
dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                               valid_pct=0.2, #80-20 train-validation random split
                               seed=999, #seed
                               label_col=1, #label is in the first column of the DataFrame
                               fn_col=0, #filename/path is in the second column of the DataFrame
                               bs=bs, #pass in batch size
                               item_tfms=item_tfms, #pass in item_tfms
                               batch_tfms=batch_tfms) #pass in batch_tfms

In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/resnet18/resnet18.pth' '/root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth'

In [ ]:
 learn = cnn_learner(dls,models.resnet18, metrics=[error_rate,accuracy], pretrained=True)

In [ ]:
os.listdir('../')

In [ ]:
learn=learn.load('../../input/model-saved/best_model_resnet18_fine_tune')

In [ ]:
learn.fine_tune(1)

In [ ]:
os.listdir('./')

In [ ]:
dataset_path = Path('../input/cassava-leaf-disease-classification')

sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
sample_df.head()
_sample_df = sample_df.copy()
_sample_df['path'] = _sample_df['image_id'].map(lambda x:dataset_path/'test_images'/x)
_sample_df = _sample_df.drop(columns=['image_id'])
_sample_df = _sample_df[['path','label']]
_sample_df.head()

In [ ]:
test_dl = learn.dls.test_dl(_sample_df)

#--

test_dl.show_batch()

#--



In [ ]:
preds, _ = learn.tta(dl=test_dl, n=8, beta=0)

sample_df['label'] = preds.argmax(dim=-1).numpy()

sample_df.to_csv('submission.csv',index=False)